In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import scipy.sparse as sp
import scipy.sparse.linalg as sppla

# Below starts the implement the 2D diffusion equation integrator

# p is the 2D -> 1D index transformation function
def p(Nx,i,j):
    return j*Nx+i

# Implicit_coeff set to 0 ===> Forward Euler, to 0.5 ===> Crank-Nicolson
Implicit_coeff = 0.0

x_left = 0.
x_right = 4.
y_bot = 0.
y_top = 4.

Lx = x_right-x_left
Ly = y_top-y_bot
Nx = 201
Ny = 201

dx = Lx/(Nx-1)
dy = Ly/(Ny-1)

x = np.linspace(x_left,x_right,num=Nx)
y = np.linspace(y_bot,y_top,num=Ny)
X, Y = np.meshgrid(x,y)

# Nt is the number of time steps taken for the integrator
Nt = 20
dt = dx*dx/4.

Fx = dt/(dx*dx)
Fy = dt/(dy*dy)


# Solving Au_next_1D = Bu_current_1D+Source_array, for u_next_1D (Assuming source is time-independent)
# Coefficient matrix A and B are purely dependent on BC

A = sp.identity(Ny*Nx)  # Coefficient matrix A for the next-step solution
B = sp.identity(Ny*Nx)  # Coefficient matrix B for the current-step values
A = sp.lil_matrix(A)
B = sp.lil_matrix(B)

u_2D = np.zeros([Ny,Nx])

IC_2D = np.zeros([Ny,Nx])
IC_array = np.zeros(Ny*Nx)

#IC_2D[int(0.4*(Ny+1)):int(0.6*(Ny+1)),int(0.4*(Nx+1)):int(0.6*(Nx+1))] = 1. #Central IC for qualitative code validation


Source_2D = np.zeros([Ny,Nx])  # Assumeing source is time-independent
Source_array = np.zeros(Ny*Nx)


Source_2D[80:100,1:10] = 1. # left source
Source_2D[160:180,1:10] = 1.

Source_2D[80:100,190:199] = 1. # right source

Source_2D[1:10,130:150] = 1. #bottom source
Source_2D[1:10,30:50] = 1.

Source_2D[190:199,95:115] = 1. # top source

Source_2D[95:105,95:115] = 1. # central source

# Begin filling up the 1D Initial condition array and source array, based on their 2D version
for j in range (0,Ny):
    for i in range (0,Nx):
        IC_array[p(Nx,i,j)] = IC_2D[j,i]
        Source_array[p(Nx,i,j)] = Source_2D[j,i]
        
# Begin making the flags matrix based on the designed geometry
on_domain = np.zeros([Ny,Nx])
on_window = np.zeros([Ny,Nx])
on_wall = np.zeros([Ny,Nx])

for i in range (0,Nx):
    for j in range (0,Ny):
        if ((i>0)and(i<Nx-1)and(j>0)and(j<Ny-1)):
            on_domain[j,i] = 1
            continue
        elif ((i==Nx-1)or(j==0)or(j==Ny-1)):
            on_wall[j,i] = 1
            continue
        elif (i==0):
            on_window[j,i] = 1
            continue
        else:
            print('Unspecifies flag on j,i=',j,i)


# Begin filling up the Coefficient matrix A for the next-step solution, and B, based on BC:
for j in range(0,Ny):
    for i in range(0,Nx):
        if (on_window[j,i]==1):
            A[p(Nx,i,j),p(Nx,i,j)] = 1  # Required by Dirchilet BC
            
            B[p(Nx,i,j),:] = 0.
            Source_array[p(Nx,i,j)] = 0.
            
        elif (on_domain[j,i]==1):
            A[p(Nx,i,j),p(Nx,i,j)] = 1 + 2*Implicit_coeff*(Fx+Fy)
            A[p(Nx,i,j),p(Nx,i-1,j)] = -1*Implicit_coeff*Fx
            A[p(Nx,i,j),p(Nx,i+1,j)] = -1*Implicit_coeff*Fx
            A[p(Nx,i,j),p(Nx,i,j-1)] = -1*Implicit_coeff*Fy
            A[p(Nx,i,j),p(Nx,i,j+1)] = -1*Implicit_coeff*Fy
            
            B[p(Nx,i,j),p(Nx,i,j)] = 1 + 2*(Implicit_coeff-1.)*(Fx+Fy)
            B[p(Nx,i,j),p(Nx,i-1,j)] = (1-Implicit_coeff)*Fx
            B[p(Nx,i,j),p(Nx,i+1,j)] = (1-Implicit_coeff)*Fx
            B[p(Nx,i,j),p(Nx,i,j-1)] = (1-Implicit_coeff)*Fy
            B[p(Nx,i,j),p(Nx,i,j+1)] = (1-Implicit_coeff)*Fy
                        
        elif (on_wall[j,i]==1):
            if (i==Nx-1):
                if (j==0):
                    A[p(Nx,i,j),p(Nx,i,j)] = 1 + 2*Implicit_coeff*(Fx+Fy)
                    A[p(Nx,i,j),p(Nx,i-1,j)] = -2*Implicit_coeff*Fx
                    A[p(Nx,i,j),p(Nx,i,j+1)] = -2*Implicit_coeff*Fy
                    
                    B[p(Nx,i,j),p(Nx,i,j)] = 1 + 2*(Implicit_coeff-1.)*(Fx+Fy)
                    B[p(Nx,i,j),p(Nx,i-1,j)] = 2*(1.-Implicit_coeff)*Fx
                    B[p(Nx,i,j),p(Nx,i,j+1)] = 2*(1.-Implicit_coeff)*Fy
                   
                elif (j==Ny-1):
                    A[p(Nx,i,j),p(Nx,i,j)] = 1 + 2*Implicit_coeff*(Fx+Fy)
                    A[p(Nx,i,j),p(Nx,i-1,j)] = -2*Implicit_coeff*Fx
                    A[p(Nx,i,j),p(Nx,i,j-1)] = -2*Implicit_coeff*Fy
                    
                    B[p(Nx,i,j),p(Nx,i,j)] = 1 + 2*(Implicit_coeff-1)*(Fx+Fy)
                    B[p(Nx,i,j),p(Nx,i-1,j)] = 2*(1.-Implicit_coeff)*Fx
                    B[p(Nx,i,j),p(Nx,i,j-1)] = 2*(1.-Implicit_coeff)*Fy
                                       
                else:
                    A[p(Nx,i,j),p(Nx,i,j)] = 1 + 2*Implicit_coeff*(Fx+Fy)
                    A[p(Nx,i,j),p(Nx,i-1,j)] = -2*Implicit_coeff*Fx
                    A[p(Nx,i,j),p(Nx,i,j-1)] = -1*Implicit_coeff*Fy
                    A[p(Nx,i,j),p(Nx,i,j+1)] = -1*Implicit_coeff*Fy
                    
                    B[p(Nx,i,j),p(Nx,i,j)] = 1 + 2*(Implicit_coeff-1)*(Fx+Fy)
                    B[p(Nx,i,j),p(Nx,i-1,j)] = 2*(1-Implicit_coeff)*Fx
                    B[p(Nx,i,j),p(Nx,i,j-1)] = (1-Implicit_coeff)*Fy
                    B[p(Nx,i,j),p(Nx,i,j+1)] = (1-Implicit_coeff)*Fy
                    
            elif (i==0):
                if (j==0):
                    A[p(Nx,i,j),p(Nx,i,j)] = 1 + 2*Implicit_coeff*(Fx+Fy)
                    A[p(Nx,i,j),p(Nx,i+1,j)] = -2*Implicit_coeff*Fx
                    A[p(Nx,i,j),p(Nx,i,j+1)] = -2*Implicit_coeff*Fy
                    
                    B[p(Nx,i,j),p(Nx,i,j)] = 1 + 2*(Implicit_coeff-1.)*(Fx+Fy)
                    B[p(Nx,i,j),p(Nx,i+1,j)] = 2*(1.-Implicit_coeff)*Fx
                    B[p(Nx,i,j),p(Nx,i,j+1)] = 2*(1.-Implicit_coeff)*Fy
                    
                elif(j==Ny-1):
                    A[p(Nx,i,j),p(Nx,i,j)] = 1 + 2*Implicit_coeff*(Fx+Fy)
                    A[p(Nx,i,j),p(Nx,i+1,j)] = -2*Implicit_coeff*Fx
                    A[p(Nx,i,j),p(Nx,i,j-1)] = -2*Implicit_coeff*Fy
                    
                    B[p(Nx,i,j),p(Nx,i,j)] = 1 + 2*(Implicit_coeff-1.)*(Fx+Fy)
                    B[p(Nx,i,j),p(Nx,i+1,j)] = 2*(1.-Implicit_coeff)*Fx
                    B[p(Nx,i,j),p(Nx,i,j-1)] = 2*(1.-Implicit_coeff)*Fy                    
                    
                else:
                    A[p(Nx,i,j),p(Nx,i,j)] = 1 + 2*Implicit_coeff*(Fx+Fy)
                    A[p(Nx,i,j),p(Nx,i+1,j)] = -2*Implicit_coeff*Fx
                    A[p(Nx,i,j),p(Nx,i,j-1)] = -1*Implicit_coeff*Fy
                    A[p(Nx,i,j),p(Nx,i,j+1)] = -1*Implicit_coeff*Fy
                    
                    B[p(Nx,i,j),p(Nx,i,j)] = 1 + 2*(Implicit_coeff-1)*(Fx+Fy)
                    B[p(Nx,i,j),p(Nx,i+1,j)] = 2*(1-Implicit_coeff)*Fx
                    B[p(Nx,i,j),p(Nx,i,j-1)] = (1-Implicit_coeff)*Fy
                    B[p(Nx,i,j),p(Nx,i,j+1)] = (1-Implicit_coeff)*Fy
                                                            
            elif (j==0):
                A[p(Nx,i,j),p(Nx,i,j)] = 1 + 2*Implicit_coeff*(Fx+Fy)
                A[p(Nx,i,j),p(Nx,i-1,j)] = -1*Implicit_coeff*Fx
                A[p(Nx,i,j),p(Nx,i+1,j)] = -1*Implicit_coeff*Fx
                A[p(Nx,i,j),p(Nx,i,j+1)] = -2*Implicit_coeff*Fy
                
                B[p(Nx,i,j),p(Nx,i,j)] = 1 + 2*(Implicit_coeff-1)*(Fx+Fy)
                B[p(Nx,i,j),p(Nx,i-1,j)] = (1-Implicit_coeff)*Fx
                B[p(Nx,i,j),p(Nx,i+1,j)] = (1-Implicit_coeff)*Fx
                B[p(Nx,i,j),p(Nx,i,j+1)] = 2*(1-Implicit_coeff)*Fy
                
            elif (j==Ny-1):
                A[p(Nx,i,j),p(Nx,i,j)] = 1 + 2*Implicit_coeff*(Fx+Fy)
                A[p(Nx,i,j),p(Nx,i-1,j)] = -1*Implicit_coeff*Fx
                A[p(Nx,i,j),p(Nx,i+1,j)] = -1*Implicit_coeff*Fx
                A[p(Nx,i,j),p(Nx,i,j-1)] = -2*Implicit_coeff*Fy
                
                B[p(Nx,i,j),p(Nx,i,j)] = 1 + 2*(Implicit_coeff-1)*(Fx+Fy)
                B[p(Nx,i,j),p(Nx,i-1,j)] = (1-Implicit_coeff)*Fx
                B[p(Nx,i,j),p(Nx,i+1,j)] = (1-Implicit_coeff)*Fx
                B[p(Nx,i,j),p(Nx,i,j-1)] = 2*(1-Implicit_coeff)*Fy
A = sp.csc_matrix(A)
B = sp.csc_matrix(B)


# Starting time of integration            
t=0
for n in range (0,Nt):
    print(n)
    if (n==0): 
        u_current_1D = IC_array # Filling up the initial condition
    else:
        u_current_1D = u_next_1D
    
    u_next_1D = sppla.spsolve(A,(B*u_current_1D) + Source_array*dt)
        
    t = t + dt
    
for j in range (0,Ny):
    for i in range (0,Nx):
        u_2D[j,i] = u_next_1D[p(Nx,i,j)]
    

plt.pcolormesh(X, Y, u_2D)
plt.xlabel(r'X (m)')
plt.ylabel(r'Y (m)')
plt.title(r'Numerical Solution (Explicit) with $\Delta x$=$\Delta y$= %g m, at T=%g s' %(dx,t))
plt.colorbar()
plt.show()

print('next step solution stores the solution at T = ',t)
print('dx =',dx)
print('dt =',dt)
print('Fx =',Fx)


In [ ]:
# Below starts to directly solve for the steady-state solution

A_SS = sp.identity(Ny*Nx)  # Coefficient matrix A for steady-state solution
A_SS = sp.lil_matrix(A_SS)

# Begin filling up the Coefficient matrix A for the next-step solution, and B, based on BC:
for j in range(0,Ny):
    for i in range(0,Nx):
        if (on_window[j,i]==1):
            A_SS[p(Nx,i,j),p(Nx,i,j)] = 1.  # Required by Dirchilet BC
            
        elif (on_domain[j,i]==1):
            A_SS[p(Nx,i,j),p(Nx,i,j)] = -2.*((1./(dx*dx))+(1./(dy*dy)))
            A_SS[p(Nx,i,j),p(Nx,i-1,j)] = 1./(dx*dx)
            A_SS[p(Nx,i,j),p(Nx,i+1,j)] = 1./(dx*dx)
            A_SS[p(Nx,i,j),p(Nx,i,j-1)] = 1./(dy*dy)
            A_SS[p(Nx,i,j),p(Nx,i,j+1)] = 1./(dy*dy)
                                                                    
        elif (on_wall[j,i]==1):
            if (i==Nx-1):
                if (j==0):
                    A_SS[p(Nx,i,j),p(Nx,i,j)] = -2.*((1./(dx*dx))+(1./(dy*dy)))
                    A_SS[p(Nx,i,j),p(Nx,i-1,j)] = 2./(dx*dx)
                    A_SS[p(Nx,i,j),p(Nx,i,j+1)] = 2./(dy*dy)

                elif (j==Ny-1):
                    A_SS[p(Nx,i,j),p(Nx,i,j)] = -2.*((1./(dx*dx))+(1./(dy*dy)))
                    A_SS[p(Nx,i,j),p(Nx,i-1,j)] = 2./(dx*dx)
                    A_SS[p(Nx,i,j),p(Nx,i,j-1)] = 2./(dy*dy)
                                       
                else:
                    A_SS[p(Nx,i,j),p(Nx,i,j)] = -2.*((1./(dx*dx))+(1./(dy*dy)))
                    A_SS[p(Nx,i,j),p(Nx,i-1,j)] = 2./(dx*dx)
                    A_SS[p(Nx,i,j),p(Nx,i,j-1)] = 1./(dy*dy)
                    A_SS[p(Nx,i,j),p(Nx,i,j+1)] = 1./(dy*dy)
                    
            elif (i==0):
                if (j==0):
                    A_SS[p(Nx,i,j),p(Nx,i,j)] = -2.*((1./(dx*dx))+(1./(dy*dy)))
                    A_SS[p(Nx,i,j),p(Nx,i+1,j)] = 2./(dx*dx)
                    A_SS[p(Nx,i,j),p(Nx,i,j+1)] = 2./(dy*dy)
                    
                elif (j==Ny-1):
                    A_SS[p(Nx,i,j),p(Nx,i,j)] = -2.*((1./(dx*dx))+(1./(dy*dy)))
                    A_SS[p(Nx,i,j),p(Nx,i+1,j)] = 2./(dx*dx)
                    A_SS[p(Nx,i,j),p(Nx,i,j-1)] = 2./(dy*dy)
                    
                else:
                    A_SS[p(Nx,i,j),p(Nx,i,j)] = -2.*((1./(dx*dx))+(1./(dy*dy)))
                    A_SS[p(Nx,i,j),p(Nx,i+1,j)] = 2./(dx*dx)
                    A_SS[p(Nx,i,j),p(Nx,i,j-1)] = 1./(dy*dy)
                    A_SS[p(Nx,i,j),p(Nx,i,j+1)] = 1./(dy*dy)
                                               
            elif (j==0):
                A_SS[p(Nx,i,j),p(Nx,i,j)] = -2.*((1./(dx*dx))+(1./(dy*dy)))
                A_SS[p(Nx,i,j),p(Nx,i-1,j)] = 1./(dx*dx)
                A_SS[p(Nx,i,j),p(Nx,i+1,j)] = 1./(dx*dx)
                A_SS[p(Nx,i,j),p(Nx,i,j+1)] = 2./(dy*dy)
                
            elif (j==Ny-1):
                A_SS[p(Nx,i,j),p(Nx,i,j)] = -2.*((1./(dx*dx))+(1./(dy*dy)))
                A_SS[p(Nx,i,j),p(Nx,i-1,j)] = 1./(dx*dx)
                A_SS[p(Nx,i,j),p(Nx,i+1,j)] = 1./(dx*dx)
                A_SS[p(Nx,i,j),p(Nx,i,j-1)] = 2./(dy*dy)
                

A_SS = sp.csc_matrix(A_SS)

u_SS_1D = sppla.spsolve(A_SS,-Source_array)

plt.plot(u_SS_1D)
plt.show()

u_SS_2D = np.zeros([Ny,Nx])
for j in range (0,Ny):
    for i in range (0,Nx):
        u_SS_2D[j,i] = u_SS_1D[p(Nx,i,j)]

plt.pcolormesh(X, Y, u_SS_2D)
plt.xlabel(r'X (m)')
plt.ylabel(r'Y (m)')
plt.title(r'Numerical Steady State Solution, with $\Delta x$=$\Delta y$=%g m' %dx)
plt.colorbar()
plt.show()


In [ ]:
# Definning the Damped Jacobi Smoother and V-cycle Solver

def Jacobi(A,b,initial_guess,Niteration):
    
    D = A.diagonal()
    D = sp.diags(D,0)
    D = sp.csc_matrix(D)

    R = A-D
    R = sp.csc_matrix(R)
    
    for n in range (0,Niteration):
        
        if (n==0):
            u_1D_current = initial_guess
        else:
            u_1D_current = u_1D_next
        
        u_1D_next = sppla.spsolve(D,b-R*u_1D_current) # next_step solution from direct Jacobi
        
    return u_1D_next

def JacobiDamp(A,b,initial_guess,Niteration):
    
    D = A.diagonal()
    D = sp.diags(D,0)
    D = sp.csc_matrix(D)

    R = A-D
    R = sp.csc_matrix(R)

    alpha = 0.05
    for n in range (0,Niteration):
        
        if (n==0):
            u_1D_current = initial_guess
        else:
            u_1D_current = u_1D_next
        
        u_1D_next = sppla.spsolve(D,b-R*u_1D_current) # next_step solution from direct Jacobi
    
        u_1D_next = alpha*u_1D_current + (1.-alpha)*u_1D_next
        
    return u_1D_next

def twolevel_Vcycle(A_h,b_h,R,P,initial_u_h):
    u_h = JacobiDamp(A_h,b_h,initial_u_h,3)
    r_h = b_h - A_h*u_h
    r_2h = R*r_h
    
    A_2h = R*A_h*P
    c_2h = sppla.spsolve(A_2h,r_2h)
    c_h = P*c_2h
    
    w_h = u_h + c_h
    
    w_h = JacobiDamp(A_h,b_h,w_h,3)
    return w_h


In [ ]:
# Below is to check the functionality of Jacobi Smoother

initial_guess = 0.25*np.random.rand(Ny*Nx)

u_2D_Jacobi = np.zeros([Ny,Nx])
u_1D_Jacobi = Jacobi(A_SS,-Source_array,initial_guess,300)

u_2D_JacobiDamp = np.zeros([Ny,Nx])
u_1D_JacobiDamp = JacobiDamp(A_SS,-Source_array,initial_guess,300)

for j in range (0,Ny):
    for i in range (0,Nx):
        u_2D_Jacobi[j,i] = u_1D_Jacobi[p(Nx,i,j)]
        u_2D_JacobiDamp[j,i] = u_1D_JacobiDamp[p(Nx,i,j)]


plt.figure()
plt.pcolormesh(X, Y, u_2D_Jacobi)
plt.xlabel(r'X (m)')
plt.ylabel(r'Y (m)')
plt.title(r'300 Jacobi Iterations, for Steady-State Solution with $\Delta x$=$\Delta y$=%g m' %dx)
plt.colorbar()
plt.show() 

plt.figure()
plt.pcolormesh(X, Y, u_2D_JacobiDamp)
plt.xlabel(r'X (m)')
plt.ylabel(r'Y (m)')
plt.title(r'300 Damped Jacobi Iterations, for Steady-State Solution with $\Delta x$=$\Delta y$=%g m' %dx)
plt.colorbar()
plt.show() 

In [ ]:
Nx_2h = int((Nx+1)/2)
Ny_2h = int((Ny+1)/2)

print('Nx_2h =', Nx_2h)
print('Ny_2h =', Ny_2h)

R = sp.lil_matrix((Nx_2h*Ny_2h,Nx*Ny))
P = sp.lil_matrix((Nx*Ny,Nx_2h*Ny_2h))

for j in range (0,Ny_2h):
    for i in range (0,Nx_2h):
        R[p(Nx_2h,i,j),p(Nx,2*i,2*j)] = 1
        
for j in range (0,Ny):
    for i in range (0,Nx):
        if ((j%2==0)and(i%2==0)):
            P[p(Nx,i,j),p(Nx_2h,int(i/2),int(j/2))] = 1.
            
        elif ((j%2==0)and(i%2==1)):
            P[p(Nx,i,j),p(Nx_2h,int(i/2),int(j/2))] = 0.5
            P[p(Nx,i,j),p(Nx_2h,int(i/2)+1,int(j/2))] = 0.5
        
        elif ((j%2==1)and(i%2==0)):
            P[p(Nx,i,j),p(Nx_2h,int(i/2),int(j/2))] = 0.5
            P[p(Nx,i,j),p(Nx_2h,int(i/2),int(j/2)+1)] = 0.5
        
        elif ((j%2==1)and(i%2==1)):
            P[p(Nx,i,j),p(Nx_2h,int(i/2),int(j/2))] = 0.25
            P[p(Nx,i,j),p(Nx_2h,int(i/2),int(j/2)+1)] = 0.25
            P[p(Nx,i,j),p(Nx_2h,int(i/2)+1,int(j/2))] = 0.25
            P[p(Nx,i,j),p(Nx_2h,int(i/2)+1,int(j/2)+1)] = 0.25
        
        else:
            print('SHOULD STOP, CHECK ELEMENT ASSIGNMENT OF P')

            
N_iteration = []
max_residual = []
initial_guess_h = 0.25*np.random.rand(Ny*Nx)

for n in range (0,100):
    print(n)
    if (n==0):
        w_h_current = initial_guess_h
    else:
        w_h_current = w_h_next
    
    w_h_next = twolevel_Vcycle(A_SS,-Source_array,R,P,w_h_current)
    
    N_iteration.append(n+1)
    max_residual.append(np.max(-Source_array-A_SS*w_h_next))

    

for j in range (0,Ny):
    for i in range (0,Nx):
        u_2D[j,i] = w_h_next[p(Nx,i,j)]


plt.figure()
plt.pcolormesh(X, Y, u_2D)
plt.colorbar()
plt.xlabel('X (m)')
plt.ylabel('Y (m)')
plt.title('Steady-State Solution After 100 Iterations of V-Cycle,with $\Delta x$=$\Delta y$=%g m' %dx)
plt.show()

plt.semilogy(N_iteration,max_residual)
plt.xlabel('Iteration Counts')
plt.ylabel('Log(Residual Norm)')
plt.title('Log of Maximum Element of Residual vs. V-Cycle Iteration Counts')
plt.show()

In [ ]:
### Below is the analytical solution used for qualitative code validation
def A(m,n,a,b,c,d,e,f):
    return 4.*(np.cos(c*m*np.pi/a)-np.cos(d*m*np.pi/a))*(np.cos(e*n*np.pi/b)-np.cos(f*n*np.pi/b))/(m*n*np.pi*np.pi)

T=1.1
u_1D_analytic = np.zeros(Ny*Nx)
u_2D_analytic = np.zeros([Ny,Nx])
for j in range (0,Ny):
    for i in range(0,Nx):
        if ((i==0)or(i==Nx-1)or(j==0)or(j==Ny-1)):
            u_2D_analytic[j,i] = 0.
        else:
            u_2D_analytic[j,i] = 0.
            for m in range(1,9):
                for n in range(1,9):
                    u_2D_analytic[j,i] = u_2D_analytic[j,i] + A(m,n,Lx,Ly,0.4*Lx,0.6*Lx,0.4*Ly,0.6*Ly)*np.sin(m*np.pi*x[i]/Lx)*np.sin(n*np.pi*y[j]/Ly)*np.exp(-T*((m*np.pi/Lx)**2 + (n*np.pi/Ly)**2))

for j in range (0,Ny):
    for i in range (0,Nx):
        u_1D_analytic[p(Nx,i,j)] = u_2D_analytic[j,i]
        
 
plt.pcolormesh(X, Y, u_2D_analytic)
plt.xlabel(r'X (m)')
plt.ylabel(r'Y (m)')
plt.title(r'Analytical Solution, at T=%g s' %T)
plt.colorbar()
plt.show()